***GENERATED CODE FOR TelcoChurn PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df

    def getTypedValue(self, type, value):
        if isinstance(type, StringType):
            return value
        elif isinstance(type, DoubleType):
            if isinstance(value, str):
                if value == '0.0':
                    return 0.0
                else:
                    return float(value)
            return value
        elif isinstance(type, IntegerType):
            if isinstance(value, str):
                if value == '0.0':
                    return 0.0
                elif "." in value:
                    return int(float(value))
                elif value.isalpha():
                    return value
                else:
                    return int(value)
            return value
        return value


Feature_Transformations_Methods = {

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(inStages, inStagesData, stageId, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = inStagesData[inStages[0]]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=15,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	TelcoChurn_DBFS = DBFSConnector.fetch([], {}, "5ea93852cf0c8a9cb7f7e106", spark, "{'url': '/Demo/TelcoCustomerChurn.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	TelcoChurn_AutoFE = TransformationMain.run(["5ea93852cf0c8a9cb7f7e106"],{"5ea93852cf0c8a9cb7f7e106": TelcoChurn_DBFS}, "5ea93852cf0c8a9cb7f7e107", spark,json.dumps( {"FE": [{"transformationsData": {"feature_label": "customerID"}, "feature": "customerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "0002-ORFBO", "max": "9993-LHIEB", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "gender"}, "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "SeniorCitizen", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4465", "mean": "0.16", "stddev": "0.37", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {"feature_label": "Partner"}, "feature": "Partner", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Dependents"}, "feature": "Dependents", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "tenure", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4465", "mean": "32.27", "stddev": "24.59", "min": "0", "max": "72", "missing": "0"}}, {"transformationsData": {"feature_label": "PhoneService"}, "feature": "PhoneService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "MultipleLines"}, "feature": "MultipleLines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "InternetService"}, "feature": "InternetService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "DSL", "max": "No", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "OnlineSecurity"}, "feature": "OnlineSecurity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "OnlineBackup"}, "feature": "OnlineBackup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "DeviceProtection"}, "feature": "DeviceProtection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "TechSupport"}, "feature": "TechSupport", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "StreamingTV"}, "feature": "StreamingTV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "StreamingMovies"}, "feature": "StreamingMovies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Contract"}, "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Two year", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "PaperlessBilling"}, "feature": "PaperlessBilling", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "PaymentMethod"}, "feature": "PaymentMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "MonthlyCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4465", "mean": "64.51", "stddev": "30.15", "min": "18.25", "max": "118.75", "missing": "0"}, "transformation": ""}, {"transformationsData": {"feature_label": "TotalCharges"}, "feature": "TotalCharges", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "2273.0", "stddev": "2268.25", "min": "100.2", "max": "999.45", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Churn"}, "feature": "Churn", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4465", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": "String Indexer"}, {"feature": "customerID_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "4465", "mean": "2232.0", "stddev": "1289.08", "min": "0.0", "max": "4464.0", "missing": "0"}}, {"feature": "gender_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "Partner_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.48", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "Dependents_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.3", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}}, {"feature": "PhoneService_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.1", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}}, {"feature": "MultipleLines_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.62", "stddev": "0.66", "min": "0", "max": "2", "missing": "0"}}, {"feature": "InternetService_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.78", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}}, {"feature": "OnlineSecurity_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.72", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "OnlineBackup_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.78", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}}, {"feature": "DeviceProtection_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.78", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}}, {"feature": "TechSupport_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.73", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "StreamingTV_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.83", "stddev": "0.76", "min": "0", "max": "2", "missing": "0"}}, {"feature": "StreamingMovies_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.82", "stddev": "0.76", "min": "0", "max": "2", "missing": "0"}}, {"feature": "Contract_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.66", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "PaperlessBilling_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"feature": "PaymentMethod_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "1.3", "stddev": "1.15", "min": "0", "max": "3", "missing": "0"}}, {"feature": "TotalCharges_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "4465", "mean": "2003.91", "stddev": "1273.38", "min": "0.0", "max": "4226.0", "missing": "0"}}, {"feature": "Churn_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "4465", "mean": "0.26", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	functionClassification(TelcoChurn_AutoFE, ["customerID", "gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges"], "Churn")

except Exception as ex: 
	logging.error(ex)
